In [1]:
import os
import math
import torch
from dataclasses import dataclass, field

from gamesopt.attacks import Attack
from gamesopt.optimizer import Optimizer
from gamesopt.aggregator import Aggregator
from gamesopt.games import Game, create_matrix, create_bias
from gamesopt.train_distributed import train

# %matplotlib widget
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [3]:
# for tag in os.environ['MLFLOW_RUN_TAG'].split():
#     print(tag)

In [4]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd()) + 'lr'

In [5]:
# class Config(BaseConfig):
@dataclass
class Config():
    n_iter: int = 6001

    n_peers: int = 20
    n_byzan: int = 4

    game: Game = Game.Quadratic
    # game: Game = field(default_factory=lambda: Game.Quadratic)
    num_samples: int = 1000
    dim: int = 10
    with_bias: bool = True
    mu: float = 1e-1
    ell: float = 1e3

    optimizer: Optimizer = None
    batch_size: int = None
    lr: float = 1/2/ell/15
    alpha: float = None
    sigmaC: float = None

    attack: Attack = None
    n_attacking: int = None
    attack_param: float = None

    use_bucketing: bool = None
    bucketing_s: int = 2

    aggregator: Aggregator = None
    aggregator_param_a: int = 10  # trimmed_mean_b=10; krum_m=2; ,clipping_n_iter=3 clipping_tau=10
    aggregator_param_b: float = 0.1 # rfa_T=10,rfa_nu=0.1    , 0.1)  # rfa_T=10,rfa_nu=0.1

In [6]:
@dataclass
class Data():
    def __init__(self, config):
        self.matrix = create_matrix(config.dim, config.num_samples,
                                    config.mu, config.ell,
                                    with_bias=config.with_bias)
        self.bias = create_bias(config.dim, config.num_samples,
                                with_bias=config.with_bias)
        self.true = torch.linalg.solve(self.matrix.mean(dim=0),
                                       -self.bias.mean(dim=0))
        self.players = self.true + .1/math.sqrt(2 * config.dim)

In [7]:
base_config = Config()
data = Data(base_config)

In [31]:
batch_size = 100
attack = Attack.ALIE

In [32]:
config = Config(batch_size=batch_size, attack=attack)
config.optimizer = Optimizer.SGDA
config.aggregator = Aggregator.Mean
config.n_attacking = config.n_byzan
os.environ['MLFLOW_RUN_TAG'] = 'RA'
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 4966
<enum 'Game'>


In [11]:
config = Config(batch_size=batch_size, attack=attack)
config.optimizer = Optimizer.SGDARA
config.aggregator = Aggregator.RFA
config.use_bucketing = True
config.n_attacking = config.n_byzan
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 22282


ProcessRaisedException: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/nazya/miniconda3/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 59, in _wrap
    fn(i, *args)
  File "/home/nazya/sgda-ra/gamesopt/train_distributed.py", line 60, in _train
    optimizer = load_distributed_optimizer(config, data, rank)
  File "/home/nazya/sgda-ra/gamesopt/train_distributed.py", line 19, in load_distributed_optimizer
    return SGDARA(config, data, rank)
  File "/home/nazya/sgda-ra/gamesopt/optimizer/distributed.py", line 42, in __init__
    self.aggregator = load_bucketing(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 29, in load_bucketing
    return Bucketing(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 37, in __init__
    self.aggregator = load_aggregator(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 24, in load_aggregator
    raise NotImp

In [41]:
config = Config(batch_size=batch_size, attack=attack)
config.optimizer = Optimizer.MSGDARA
config.use_bucketing = True
config.n_attacking = config.n_byzan
config.alpha = 0.1
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 33784


In [34]:
config = Config(batch_size=batch_size, attack=attack)
config.n_iter = int(config.n_iter / 2) + 1
config.n_attacking = config.n_byzan
config.optimizer = Optimizer.RDEG
config.aggregator = Aggregator.UnivariateTM
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 13788
<enum 'Game'>


In [33]:
config = Config(batch_size=batch_size, attack=attack)
config.n_iter = int(config.n_iter / 2) + 1
config.optimizer = Optimizer.SEGRA
config.use_bucketing = True
config.n_attacking = config.n_byzan
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 35030
<enum 'Game'>


ProcessRaisedException: 

-- Process 2 terminated with the following error:
Traceback (most recent call last):
  File "/home/nazya/miniconda3/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 59, in _wrap
    fn(i, *args)
  File "/home/nazya/sgda-ra/gamesopt/train_distributed.py", line 61, in _train
    optimizer = load_distributed_optimizer(config, data, rank)
  File "/home/nazya/sgda-ra/gamesopt/train_distributed.py", line 23, in load_distributed_optimizer
    return SEGRA(config, data, rank)
  File "/home/nazya/sgda-ra/gamesopt/optimizer/distributed.py", line 112, in __init__
    self.aggregator = load_bucketing(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 29, in load_bucketing
    return Bucketing(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 37, in __init__
    self.aggregator = load_aggregator(config)
  File "/home/nazya/sgda-ra/gamesopt/aggregator/__init__.py", line 24, in load_aggregator
    raise NotImp

In [49]:
config = Config(batch_size=batch_size, attack=attack)
config.n_iter = config.n_iter * 2 - 1
config.optimizer = Optimizer.SGDACC
config.aggregator = Aggregator.Mean
config.n_attacking = 1
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 22342


In [50]:
config = Config(batch_size=batch_size, attack=attack)
config.n_iter = config.n_iter + 1
config.optimizer = Optimizer.SEGCC
config.aggregator = Aggregator.Mean
config.use_bucketing = False
config.n_attacking = 1
os.environ['MLFLOW_RUN_TAG'] = ''
os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
os.environ['MLFLOW_RUN_TITLE'] = ''
train(config, data)

Trying port 11461
